This notebook is a template to prepare the input aperture corners for apodised photometry. This means you will likely need to adapt the code for your own purposes.

You will need to provide a table of apertures with columns "RA_i", "Dec_i" in degrees for each ith corner of the aperture, as well as a column with the source "ID". If multiple observations are made for each ID, you can optionally only list the ID in the first instance, provided each observation is grouped together.

You will also need to provide a table of each source with "RA", "Dec" and "ID" columns.

You may find commands like ":%s/\<search\>/\<replace\>/g" in Vim or "esc+x; string-replace; \<search\>; \<replace\>" in eMACS useful in preparing these files.

In [13]:
print("Preparing environment...")

import astropy.table as tbl
import numpy as np

print("Done.")

Preparing environment...
Done.


In [16]:
aperture_table = tbl.Table.read(
    "../../SharedData/NIRSpec/MEDIUM/ds9_slit_regions.dat", format="ascii.csv"
)
source_table = tbl.Table.read(
    "../../SharedData/NIRSpec/MEDIUM/ds9_targ_regions.dat", format="ascii.csv"
)
# for x in range(0, len(aperture_table), 5):
#     for y in range(1, 5, 1):
#         aperture_table["ID"][x+y] = aperture_table["ID"][x]
output_table = tbl.join(aperture_table, source_table, keys="ID")

In [17]:
output_table.write('../../SharedData/NIRSpec/MEDIUM/medium_apertures.fits', overwrite=True)

In [18]:
output_table

RA_1,Dec_1,RA_2,Dec_2,RA_3,Dec_3,RA_4,Dec_4,ID,Source_RA,Source_Dec
float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
53.1462454,-27.769601,53.1460986,-27.7696078,53.1461027,-27.7696633,53.1462495,-27.7696565,4028,53.1461433,-27.7696165
53.1540719,-27.7704383,53.1539245,-27.7704451,53.1539286,-27.7705007,53.1540761,-27.7704939,4081,53.1540398,-27.7704472
53.1522122,-27.761218,53.1520649,-27.7612251,53.1520689,-27.7612807,53.1522162,-27.7612735,7236,53.1521909,-27.7612526
53.1561388,-27.7617039,53.1559912,-27.761711,53.1559953,-27.7617666,53.1561429,-27.7617595,7511,53.1560982,-27.7617165
53.1433424,-27.7713108,53.1431958,-27.7713176,53.1431999,-27.7713731,53.1433465,-27.7713663,8109,53.1432748,-27.7713267
53.1483937,-27.7713018,53.1482468,-27.7713086,53.1482509,-27.7713641,53.1483978,-27.7713573,9141,53.1483778,-27.7713067
53.1186092,-27.7708952,53.1184645,-27.770902,53.1184685,-27.7709572,53.1186132,-27.7709504,12127,53.1185194,-27.7709408
...,...,...,...,...,...,...,...,...,...,...
53.0952701,-27.760587,53.0951271,-27.7605941,53.0951309,-27.760649,53.0952739,-27.7606419,58850,53.095171,-27.760609
